# Conv Net on Media Eval 2015 Placing Task - Locale

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
# from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver

# import own modules
import data_utils

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

run the following from the cs231n directory and try again:
python setup.py build_ext --inplace
You may also need to restart your iPython kernel


In [4]:
SMALL_000_DATA_X_FILE = 'data_maps/x_input_000_small.npy'
SMALL_000_DATA_Y_FILE = 'data_maps/y_country_name_000_small.npy'

In [5]:
X = data_utils.load_npy_file(SMALL_000_DATA_X_FILE)
Y = data_utils.load_npy_file(SMALL_000_DATA_Y_FILE)
print 'X: {}'.format(X.shape)
print 'Y: {}'.format(Y.shape)
print 'Y sample ', Y[:10]

X: (1876, 300, 500, 3)
Y: (1876,)
Y sample  [0 1 1 2 3 2 4 1 0 2]


In [6]:
num_samples, W, H, C = X.shape

NUM_CLASSES = 5

In [7]:
#Splitting into train, val, test sets

num_train = int(num_samples * 0.8)
num_val = int(num_samples * 0.1)
num_test = num_samples - num_train - num_val

print 'num_train %d, num_val %d, num_test %d' % (num_train, num_val, num_test)

X_train = X[:num_train]
X_val = X[num_train:num_train+num_val]
X_test = X[num_train+num_val:]

Y_train = Y[:num_train]
Y_val = Y[num_train:num_train+num_val]
Y_test = Y[num_train+num_val:]

print 'X_train %s, X_val %s, X_test %s' % (X_train.shape, X_val.shape, X_test.shape)
print 'Y_train %s, Y_val %s, Y_test %s' % (Y_train.shape, Y_val.shape, Y_test.shape)


num_train 1500, num_val 187, num_test 189
X_train (1500, 300, 500, 3), X_val (187, 300, 500, 3), X_test (189, 300, 500, 3)
Y_train (1500,), Y_val (187,), Y_test (189,)


## Overfit Small Data

In [ ]:
num_train = 100
small_data = {
  'X_train': X_train[:num_train],
  'y_train': Y_train[:num_train],
  'X_val': X_val[:num_train],
  'y_val': Y_val[:num_train],
}

model = ThreeLayerConvNet(input_dim = (C, W, H), num_classes = 5, weight_scale=1e-2)

solver = Solver(model, small_data,
                num_epochs=2, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-5,
                },
                verbose=True, print_every=1)
solver.train()

N, W, H, C (50, 300, 500, 3)
F, _, HH, WW (32, 3, 7, 7)
weights shape = 